# **Setup Environment**

In [ ]:
# Change directory to the project root so that relative paths in .env
# (e.g., model weights, data directories) resolve correctly
%cd ../

In [ ]:
import sys
sys.path.append('.')

import json
import requests
from app.rest.config import rest_config

In [ ]:
# Start MinIO — required by code that interacts with object storage
!pixi run just minio-up

In [ ]:
# # After finishing your work, shut down MinIO to free resources
# !pixi run just minio-down

In [ ]:
# Change directory to 'assets/' so that files can be referenced directly 
# (e.g., 'test.png' instead of 'assets/test.png')
%cd playground-testing/assets/

## **Test**

Start the FastAPI server — choose ONE of the following based on your hardware:

```bash
pixi run start-api-gpu   # Use if you have a CUDA-compatible GPU
pixi run start-api-cpu   # Use for CPU-only inference
```

After the server starts successfully, you’ll see a message like:  
`INFO:     Uvicorn running on http://0.0.0.0:8000`

Open your browser and navigate to **http://localhost:8000/docs** to access the interactive Swagger UI, where you can explore and test all available API endpoints.

> **Important:**  
Do NOT run these commands directly in this Jupyter notebook  as magic commands because they will block the kernel (the server runs indefinitely), preventing you from executing any further cells. Instead, run the server in a separate terminal, then use this notebook to send requests.

> **Note:**  
This notebook is intended only for testing GET endpoints that return downloadable content, as such responses cannot be properly previewed or verified directly in Swagger UI.

In [ ]:
def test_download_analysis_endpoint(video_id: str, output_path: str) -> None:
    """
    Test the analysis download endpoint and save the response to a JSON file.

    This function sends an HTTP GET request to the `/storage/analysis/{video_id}/download`
    endpoint of the running REST service, validates the response, and saves the resulting
    JSON payload to the specified local file path.

    Parameters
    ----------
    video_id : str
        UUID4 string identifying the video whose analysis should be downloaded.
    output_path : str or Path
        Local filesystem path where the downloaded analysis JSON will be saved.
    """

    url = f"http://{rest_config.host}:{rest_config.port}/{rest_config.storage_router_prefix}/analysis/{video_id}/download"

    response = requests.get(url, timeout=10)
    response.raise_for_status()

    data = response.json()
    if not isinstance(data, dict):
        raise ValueError("Response body is not a JSON object")

    # Save to file
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"Successfully downloaded analysis for video_id='{video_id}'")

In [ ]:
test_download_analysis_endpoint(
    video_id = "ee20421a-860e-4185-9ff4-5073a7c659eb", 
    output_path = "5.analysis.json"
)

In [ ]:
def test_download_video_endpoint(video_id: str, output_path: str) -> None:
    """
    Test the video download endpoint and save the response to a local video file.

    This function sends an HTTP GET request to the `/storage/v1/video/{video_id}/download`
    endpoint of the running REST service and saves the streamed video content to the
    specified local file path.

    Parameters
    ----------
    video_id : str
        UUID4 string identifying the video to download.
    output_path : str
        Local filesystem path where the downloaded video file will be saved.
    """

    url = f"http://{rest_config.host}:{rest_config.port}/storage/v1/video/{video_id}/download"

    response = requests.get(url, timeout=30, stream=True)
    response.raise_for_status()

    with open(output_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

    print(f"Successfully downloaded video for video_id='{video_id}'")

In [ ]:
test_download_video_endpoint(
    video_id = "ee20421a-860e-4185-9ff4-5073a7c659eb", 
    output_path = "5.video.mp4"
)